In [ ]:
import pandas as pd 
import numpy as np 
import nltk 
nltk.download('punkt')
from nltk import word_tokenize , sent_tokenize
from sklearn.model_selection import train_test_split

import tensorflow as tf 
import tensorflow.keras as keras 
from tensorflow.keras.layers import Dense , LSTM , Embedding , Dropout
from keras import Sequential 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data1 = pd.read_csv(r"/kaggle/input/content-generation/cloze_test_test__winter2018-cloze_test_ALL_test - 1.csv")
#data2 = pd.read_csv(r"/kaggle/input/content-generation/cloze_test_val__winter2018-cloze_test_ALL_val - 1 - 1.csv")

In [ ]:
def getsentences(dataframe):
    
    input_sentences = dataframe['InputSentence1'] + dataframe['InputSentence2'] + dataframe['InputSentence3'] + dataframe['InputSentence4'] 
    
    return input_sentences , dataframe['RandomFifthSentenceQuiz1'] , dataframe['RandomFifthSentenceQuiz2']

In [ ]:
def preprocess(text):
    
    text = text.lower()
    text = text.strip()
    
    sentences = sent_tokenize(text)
    #words = []
    #for sentence in sentences:
     #   word_list = word_tokenize(sentence)
      #  words += word_list 
    
    return sentences

In [ ]:
def proc2(y):
    y__ = []
    for i in y:
        for j in i.split():
            if j not in y__:
                y__.append(j)
    
    return y__

In [ ]:
input_df1 , out1_1 , out2_1 = getsentences(data1)
#input_df2 , out1_2 , out2_2 = getsentences(data2)

In [ ]:
input_df1['words'] = input_df1.apply(preprocess)
#input_df2['words'] = input_df2.apply(preprocess)

In [ ]:
out1_1["words"] = out1_1.apply(preprocess)
#out1_2["words"] = out1_2.apply(preprocess)

out2_1["words"] = out2_1.apply(preprocess)
#out2_2["words"] = out2_2.apply(preprocess)


In [ ]:
input_list = []
output_list = []

In [ ]:
for words in input_df1['words']:
    input_list.append(words)

#for words in input_df2['words']:
 #   input_list.append(words)

for words in out1_1['words']:
    input_list.append(words)

#for words in out1_2['words']:
  #  input_list.append(words)


#for words in out2_1['words']:
 #   input_list.append(words)

#for words in out2_2['words']:
    #input_list.append(words)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([word_tokenize(i[0]) for i in input_list] )
sequences = tokenizer.texts_to_sequences([word_tokenize(i[0]) for i in input_list])

In [ ]:
sequences[0]

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
max_len = 100

In [ ]:
embedding_dim = 100
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    LSTM(64, return_sequences=True),
    LSTM(64 , return_sequences = True),
    Dense(vocab_size, activation='softmax')
])

In [ ]:
dataframe = data1.copy()

In [ ]:
x = dataframe['InputSentence1'] + dataframe['InputSentence2'] + dataframe['InputSentence3'] + dataframe['InputSentence4'] 
y = dataframe['RandomFifthSentenceQuiz1']

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 42)

In [ ]:
x_train_list = []
x_test_list = []
y_train_list = []
y_test_list = []

In [ ]:
for i in x_train:
    x_train_list.append([i])

In [ ]:
for i in x_test:
    x_test_list.append([i])

In [ ]:
for i in y_train:
    y_train_list.append([i])

In [ ]:
for i in y_test:
    y_test_list.append([i])

In [ ]:
x_train_seq = tokenizer.texts_to_sequences([word_tokenize(i[0]) for i in x_train_list])
x_test_seq = tokenizer.texts_to_sequences([word_tokenize(i[0]) for i in x_test_list])

y_train_seq = tokenizer.texts_to_sequences([word_tokenize(i[0]) for i in y_train_list])
y_test_seq = tokenizer.texts_to_sequences([word_tokenize(i[0]) for i in y_test_list])


In [ ]:
max_pad = 100

In [ ]:
x_train_seq_padded = pad_sequences(x_train_seq ,  maxlen = max_pad)
x_test_seq_padded = pad_sequences(x_test_seq ,  maxlen = max_pad)

y_train_seq_padded = pad_sequences(y_train_seq ,  maxlen = max_pad)
y_test_seq_padded = pad_sequences(y_test_seq ,  maxlen = max_pad)

In [ ]:
x_train_seq_padded[1]

In [ ]:
#y_train_one_hot_enc = tf.keras.utils.to_categorical(y_train_seq_padded , vocab_size)

In [ ]:
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam')

In [ ]:
with tf.device('gpu:0'):
    model.fit(x_train_seq_padded , tf.one_hot(y_train_seq_padded , vocab_size) , epochs = 25 , batch_size = 16)

In [ ]:
with tf.device('gpu:0'):
    model.fit(x_train_seq_padded , tf.one_hot(y_train_seq_padded , vocab_size) , epochs = 25 , batch_size = 16)

In [ ]:
with tf.device('gpu:0'):
    model.fit(x_train_seq_padded , tf.one_hot(y_train_seq_padded , vocab_size) , epochs = 25 , batch_size = 16)

In [ ]:
with tf.device('gpu:0'):
    model.fit(x_train_seq_padded , tf.one_hot(y_train_seq_padded , vocab_size) , epochs = 25 , batch_size = 16)

In [ ]:
y_pred = model.predict(x_test_seq_padded)

In [ ]:
tokenizer.sequences_to_texts(x_test_seq_padded)[0]

In [ ]:
tokenizer.sequences_to_texts(y_test_seq_padded)[0]

In [ ]:
output = proc2(tokenizer.sequences_to_texts(np.argmax(y_pred , axis = 1)))

In [ ]:
" ".join(output)

In [ ]:
a = ["James has played baseball for eight years.	This year James is invited to take guitar lessons after school.	Guitar lessons are on the same day as baseball practice.James chooses not to play baseball this year."]

In [ ]:
x = []
for i in a:
    x.append([i])

In [ ]:
x

In [ ]:
tok = tokenizer.texts_to_sequences([word_tokenize(i[0]) for i in x])

In [ ]:
x_ = pad_sequences(tok , max_pad)

In [ ]:
y_ = (tokenizer.sequences_to_texts(np.argmax(model.predict(x_) , axis = 1)))

In [ ]:
 " ".join(y_)